## Supervised Learning

### Predicting orientation with K-Nearest Neighbors

In [1]:
#Importing all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [27]:
#reload the dataset
profiles = pd.read_csv(r"C:\Users\ntama\Documentos\Data Science\Codecademy\projects\Data for dating app\profiles.csv")
#dropping essay-type questions for this model
profiles = profiles.drop([f'essay{i}' for i in range(10)], axis=1)

#dropping 'income' since it has 81% missing values
profiles = profiles.drop(['income'], axis=1)

#changing 'ethnicity' with all values for 'ethnicity_base' from notebook 01
ethnicity_base = pd.read_csv('ethnicity_base.csv')

profiles = profiles.reset_index(drop=True)
ethnicity_base = ethnicity_base.reset_index(drop=True)
profiles['ethnicity_base'] = ethnicity_base.iloc[:, 0]

#dropping current ethnicity column and last_online
profiles = profiles.drop(['ethnicity', 'last_online'], axis=1)

#dropping 'speaks' because it has 7900 unique values
profiles = profiles.drop(['speaks'], axis=1)

profiles.head()


,age,body_type,diet,drinks,drugs,education,height,job,location,offspring,orientation,pets,religion,sex,sign,smokes,status,ethnicity_base
0,22,a little extra,strictly anything,socially,never,working on college/university,75.0,transportation,"south san francisco, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,single,"asian, white"
1,35,average,mostly other,often,sometimes,working on space camp,70.0,hospitality / travel,"oakland, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,single,white
2,38,thin,anything,socially,NaN,graduated from masters program,68.0,NaN,"san francisco, california",NaN,straight,has cats,NaN,m,pisces but it doesn&rsquo;t matter,no,available,unknown
3,23,thin,vegetarian,socially,NaN,working on college/university,71.0,student,"berkeley, california",doesn&rsquo;t want kids,straight,likes cats,NaN,m,pisces,no,single,white
4,29,athletic,NaN,socially,never,graduated from college/university,66.0,artistic / musical / writer,"san francisco, california",NaN,straight,likes dogs and likes cats,NaN,m,aquarius,no,single,mixed ethnicity


In [28]:
for column in profiles.columns:
    print(f"Unique values for {column}: {profiles[column].unique()}")
    print(f"{len(profiles[column].unique())}")

Unique values for age: [ 22  35  38  23  29  32  31  24  37  28  30  39  33  26  27  20  25  40
  36  21  34  43  46  41  42  45  18  55  50  59  44  48  54  51  62  52
  19  58  66  53  63  47  49  61  60  57  56  65  64  68 110  69  67 109]
54
Unique values for body_type: ['a little extra' 'average' 'thin' 'athletic' 'fit' nan 'skinny' 'curvy'
 'full figured' 'jacked' 'rather not say' 'used up' 'overweight']
13
Unique values for diet: ['strictly anything' 'mostly other' 'anything' 'vegetarian' nan
 'mostly anything' 'mostly vegetarian' 'strictly vegan'
 'strictly vegetarian' 'mostly vegan' 'strictly other' 'mostly halal'
 'other' 'vegan' 'mostly kosher' 'strictly halal' 'halal'
 'strictly kosher' 'kosher']
19
Unique values for drinks: ['socially' 'often' 'not at all' 'rarely' nan 'very often' 'desperately']
7
Unique values for drugs: ['never' 'sometimes' nan 'often']
4
Unique values for education: ['working on college/university' 'working on space camp'
 'graduated from masters progr